In [3]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')

# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist() #Not survivorship bias free

# end_date = '2024-05-11'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list,
#                  start=start_date,
#                  end=end_date).stack()

# df.index.names = ['date', 'ticker']
# df.columns = df.columns.str.lower()

# #Calculating features and technical indicators for each stock

# #Garmen-Klass volatility
# df['garmen_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# #RSI
# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,0])
# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,1])
# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,2])

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2022-05-12 A       114.532455  116.129997  116.250000  112.779999  113.610001   
           AAL      15.490000   15.490000   16.129999   15.090000   15.710000   
           AAPL    140.965164  142.559998  146.199997  138.800003  142.770004   
           ABBV    142.735245  154.289993  154.350006  150.149994  153.380005   
           ABNB    115.940002  115.940002  120.489998  111.220001  113.660004   
...                       ...         ...         ...         ...         ...   
2024-05-10 XYL     143.000000  143.000000  143.619995  142.300003  142.880005   
           YUM     137.619995  137.619995  139.649994  136.380005  138.149994   
           ZBH     121.309998  121.309998  122.220001  120.989998  121.389999   
           ZBRA    315.799988  315.799988  319.079987  314.200012  319.040009   
           ZTS     169.039993  169.039993  169.990005  168.289993  168.710007   

Price                   volume  garmen_klass_vol        rsi    bb_low  \
date       ticker                                                       
2022-05-12 A         1338000.0          0.000434        NaN       NaN   
           AAL      40155400.0          0.002144        NaN       NaN   
           AAPL    182602000.0          0.001286        NaN       NaN   
           ABBV      6914800.0         -0.001618        NaN       NaN   
           ABNB      9264300.0          0.003052        NaN       NaN   
...                        ...               ...        ...       ...   
2024-05-10 XYL        704100.0          0.000042  77.224583  4.810736   
           YUM       2066200.0          0.000275  50.224201  4.894712   
           ZBH        772700.0          0.000051  46.379145  4.773831   
           ZBRA       304100.0          0.000079  61.523837  5.565538   
           ZTS       2010700.0          0.000049  54.591002  4.962191   

Price                bb_mid   bb_high  
date       ticker                      
2022-05-12 A            NaN       NaN  
           AAL          NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABNB         NaN       NaN  
...                     ...       ...  
2024-05-10 XYL     4.890812  4.970888  
           YUM     4.929981  4.965251  
           ZBH     4.794068  4.814306  
           ZBRA    5.689781  5.814023  
           ZTS     5.061069  5.159946  

[250817 rows x 11 columns]